# <font color='red'>Implement CNN from scratch</font>

### In the following assignment you have to compute the convolutional operations from scratch using Numpy
You will then verify the correctness of the your implementation using a "grader" function/cell (provided by us) which will match your implmentation.

The grader fucntion would help you validate the correctness of your code. 

Please submit the final Colab notebook in the classroom ONLY after you have verified your code using the grader function/cell.


**NOTE: DO NOT change the "grader" functions or code snippets written by us.Please add your code in the suggested locations.**

Ethics Code:
1. You are welcome to read up online resources to implement the code. 
2. You can also discuss with your classmates on the implmentation over Slack.
3. But, the code you write and submit should be yours ONLY. Your code will be compared against other stduents' code and online code snippets to check for plagiarism. If your code is found to be plagiarised, you will be awarded zero-marks for all assignments, which have a 10% weightage in the final marks for this course.

### Task 1. Calcuate the shape of the output image
We use the below formula to calculate the shape of output image after performing the required operations
<img src='https://i.imgur.com/BlepuHW.jpg'>

In [1]:
#https://stats.stackexchange.com/questions/351115/convolution-with-a-non-square-kernel
#for simplicity we are assuming stride=1
import numpy as np
def calculate_output_shape(img_size, kernel_size,padding):
    #
    img_length=img_size[0]
    img_width=img_size[1]
    kernel_length=kernel_size[0]
    kernel_width=kernel_size[1]
    #return -1,-1 if the shape of kernel is greater than img_size
    if kernel_length>img_length or kernel_width>img_width:
      return -1,-1
    output_length = (img_length+(2*padding)-kernel_length)+1
    output_width = (img_width+(2*padding)-kernel_width)+1
    
    #use the above formula to calculate output_length and output_width
    
    return output_length,output_width

In [2]:
def grader_calculate_size(l1,w1,l2,w2,l3,w3):
    case_1=(l1==220 and w1==220)
    case_2=(l2==-1 and w2==-1)
    case_3=(l3==128 and w3==128)
    assert(case_1 and case_2 and case_3)
    return True
l1,w1=calculate_output_shape((224,224), (5,5),0)
l2,w2=calculate_output_shape((10,10),(12,12),1)
l3,w3=calculate_output_shape((128,128), (3,3),1)
grader_calculate_size(l1,w1,l2,w2,l3,w3)

True

### Task 2 . Get the padded image
In this task we have to add layer of zeros as padding layers in the original image

In [3]:
def get_padded_image(input_img, padding_width):
    
    #if the padding length is less than or equal to 0 then return the original image
    if(padding_width<=0):
        return img
    img_with_padding = np.zeros((input_img.shape[0]+ 2*padding_width,input_img.shape[1] + 2*padding_width))
    #you can initialize a matrix of all zeros and then copy the image data 
    for i in range(input_img.shape[0]):
      for j in range(input_img.shape[1]):
          img_with_padding[i+padding_width,j+padding_width] = input_img[i,j]

    return img_with_padding

In [4]:
def grader_padding(img1,img2):
    a1=((img1.shape)==(8,8))
    a2=((img2.shape)==(4,4))
    assert(a1 and a2)
    b1=(img1.sum(axis=1)==[ 0.,  0., 10., 26., 42., 58.,  0.,  0.])
    b2=(img1.sum(axis=0)==[ 0, 0, 28, 32, 36, 40,  0,  0])
    assert(b1.all() and b2.all())
    return True

img=np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
img1=get_padded_image(img, 2)
img2=get_padded_image(img, 0)
grader_padding(img1,img2)

True

### Task 3 . Performing convolution
In this task we have to add layer of zeros as padding layers in the original image

In [5]:
def convolve(img, kernel,pad):
    #calcualte the size of required image
    tgt_size_length,tgt_size_width =  calculate_output_shape((img.shape[0],img.shape[1]),(kernel.shape[0],kernel.shape[1]),pad)
    
    #initialize the matrix of all zeros
    convolved_img = np.zeros(shape=(tgt_size_length, tgt_size_width))
    new_img=get_padded_image(img, pad)
    # write the code for convolution

    list = []
    for i in range(new_img.shape[0]-1):
      for j in range(new_img.shape[1]-1):
        Z = new_img[i:kernel.shape[0]+i,j:j+kernel.shape[1]]
        val = 0
        for x in range(kernel.shape[0]):
          for y in range(kernel.shape[1]):
            val+=Z[x][y]*kernel[x][y]
        list.append(val)
    convolved_img = np.array(list).reshape(tgt_size_length,tgt_size_width)
    return convolved_img

In [6]:
def grader_convolve(conv_img):
    a1=(conv_img.shape==(5,5))
    a2=(conv_img.sum(axis=1)==([10, 26, 42, 58,  0]))
    a3=(conv_img.sum(axis=0)==([ 0,28, 32, 36, 40]))
    assert(a1 and a2.all() and a3.all())
    return True
img=np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
kernel=np.array([[1,-1],[0,1]])
conv_img=convolve(img, kernel,1)
grader_convolve(conv_img)

True

### Task 4: Perform Maxpooling with strides


In [7]:
def get_maxpools(img, pool_size, stride):
    # write your code to get the pooled matrix
    list = []
    for i in range(img.shape[0]-stride):
      if i != 0:
        i=i+stride-1
      for j in range(img.shape[1]-stride):
        if j != 0:
          j=j+stride-1
        pooling_mat = img[i:pool_size+i,j:j+pool_size]
        pooling_mat = pooling_mat.reshape(2*pool_size)
        list.append(max(pooling_mat))
    tgt_length = ((img.shape[0] - pool_size)/stride) + 1
    tgt_width = ((img.shape[1] - pool_size)/stride) + 1
    pooling_mat = np.array(list).reshape(int(tgt_length),int(tgt_width))
    return (pooling_mat)

In [8]:
def pooling_final(pooled_output):
    a1=(pooled_output.sum(axis=0)==[30, 33, 36])
    a2=(pooled_output.sum(axis=1)==[21, 33, 45])
    assert(a1.all() and a2.all())
    return True
img=np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
pool_size=2
stride=1
pooled_output=(get_maxpools(img, pool_size, stride))
pooling_final(pooled_output)

True